In [1]:
from utils import get_data
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
data = get_data('../data')

In [3]:
def encodeColumn(df, column, providedList=None):
    if(providedList == None):
        providedList = list(df[column].unique())
    notInListValue = len(providedList)
    return [providedList.index(x) if x in providedList else notInListValue for x in df[column]]

def convertDate(date, dateFormat='%m/%d/%Y'):
    if(isinstance(date, str)):
        date = (datetime.strptime(date, dateFormat)).toordinal()
    return date

In [4]:
data['PROPTYPE_CAT'] = encodeColumn(data, 'PROPTYPE', ['SF', 'CC', 'MF'])
print(data[['PROPTYPE','PROPTYPE_CAT']].head(20))

data['STYLE_CAT'] = encodeColumn(data, 'STYLE')
print(data[['STYLE','STYLE_CAT']].head(20))

data['ZIP_CAT'] = encodeColumn(data, 'ZIP')
print(data[['ZIP','ZIP_CAT']].head(20))

data['LISTDATE_CAT'] = [convertDate(x) for x in data['LISTDATE']]
data['SOLDDATE_CAT'] = [convertDate(x) for x in data['SOLDDATE']]
print(data[['LISTDATE', 'LISTDATE_CAT']].head(20))

print(data[data['STATUS'] == 'SLD'][['SOLDDATE', 'SOLDDATE_CAT']].head())

   PROPTYPE  PROPTYPE_CAT
0        SF             0
1        SF             0
2        SF             0
3        SF             0
4        SF             0
5        SF             0
6        SF             0
7        SF             0
8        SF             0
9        SF             0
10       SF             0
11       SF             0
12       SF             0
13       SF             0
14       SF             0
15       SF             0
16       SF             0
17       SF             0
18       SF             0
19       SF             0
                            STYLE  STYLE_CAT
0             Other (See Remarks)          0
1                            Cape          1
2                        Bungalow          2
3                        Bungalow          2
4                        Colonial          3
5                        Colonial          3
6                        Colonial          3
7                        Colonial          3
8                        Colonial          3
9   

# Define Criteria

Find home that have been sold, and remove condos since they the units may have the same address, and trying to match on address. Note this means we will **not** be working with any condo data in this project, since it is hard to identify flipped condos.

We then match to find houses with the same street address.

In [5]:
soldHomes = data[data['STATUS'] == 'SLD']
soldHomes = soldHomes[soldHomes['PROPTYPE'] != 'CC']

In [6]:
soldMLS = soldHomes['MLSNUM'].unique()

In [7]:
# find home with same address
def getHome(data, mls):
    home = soldHomes[soldHomes['MLSNUM'] == mls]
    homes = soldHomes[soldHomes['ZIP'] == home['ZIP'].iloc[0]]
    homes = homes[homes['HOUSENUM1'] == home['HOUSENUM1'].iloc[0]]
    homes = homes[homes['STREETNAME'] == home['STREETNAME'].iloc[0]]
    return homes

# Find Potential Flips

Go through all mls listing to find matching homes.

Remaining homes (ones that are not flips) will end up on the control/non-flippable list.

In [8]:
test_homes = []
control_homes = []

for mls in soldMLS:
    homes = getHome(soldHomes, mls)
    if(homes.shape[0] == 2):
        test_homes.append(homes)
    else:
        control_homes.append(homes)
        

In [9]:
flippedHomes = pd.concat(test_homes)
print(flippedHomes.shape)

controlHomes = pd.concat(control_homes)
print(controlHomes.shape)

(9770, 43)
(126081, 43)


In [10]:
# remove duplicates and sort by mls
flippedHomes = flippedHomes.drop_duplicates(subset='MLSNUM', keep="first")
flippedHomes = flippedHomes.sort_values(by='MLSNUM')

controlHomes = controlHomes.drop_duplicates(subset='MLSNUM', keep="first")

# Build Test and Control Data Sets

Create the control data set from our array of control homes. Then, put all the pre-flipped homes in as "flippable", and the post-flipped homes back int othe control pile (as they are no longer flippable).

In [11]:
# build flip set
flip_data = [];
post_flips = [];
for mls in flippedHomes['MLSNUM']:
    homes = getHome(flippedHomes, mls)
    # check if home has 2 mls rows
    if(homes.shape[0] >= 2):
        pre = homes.iloc[0]
        post = homes.iloc[1]
        # check if it's not the same mls number
        if pre['MLSNUM'] == post['MLSNUM']:
            continue
        # find pre and post flip values
        if pd.to_datetime(pre['SOLDDATE']) > pd.to_datetime(post['SOLDDATE']):
            # post < pre: swap them
            swap = pre
            pre = post
            post = swap
        flip_data.append({
            'FLIPPABLE': "1",
            'MLSNUM': pre['MLSNUM'],
            'SOLDPRICE': pre['SOLDPRICE'],
            'DOM': pre['DOM'], #days on market
            'ZIP': pre['ZIP'],
            'BEDS': pre['BEDS'],
            'BATHS': pre['BATHS'],
            'SQFT': pre['SQFT'],
            'AGE': pre['AGE'],
            'LOTSIZE': pre['LOTSIZE'],
            'GARAGE': pre['GARAGE'],
            'PROPTYPE_CAT': pre['PROPTYPE_CAT'],
            'STYLE_CAT': pre['STYLE_CAT'],
            'ZIP_CAT': pre['ZIP_CAT'],
            'SOLDDATE_CAT': pre['SOLDDATE_CAT'],
        })
        post_flips.append({ # we want to add our post-flip data to the control, since these are not flippable anymore
            'FLIPPABLE': "0",
            'MLSNUM': post['MLSNUM'],
            'SOLDPRICE': post['SOLDPRICE'],
            'DOM': post['DOM'],
            'BEDS': post['BEDS'],
            'BATHS': post['BATHS'],
            'SQFT': post['SQFT'],
            'AGE': post['AGE'],
            'GARAGE': post['GARAGE'],
            'PROPTYPE_CAT': post['PROPTYPE_CAT'],
            'STYLE_CAT': post['STYLE_CAT'],
            'ZIP_CAT': post['ZIP_CAT'],
            'SOLDDATE_CAT': post['SOLDDATE_CAT'],
        })

In [12]:
# build control set
post_flips = pd.DataFrame.from_dict(post_flips)
print(post_flips.shape)

unflipped_homes = controlHomes[['MLSNUM','SOLDPRICE','DOM','BEDS','BATHS','SQFT','AGE','GARAGE',
                                'PROPTYPE_CAT','STYLE_CAT','ZIP_CAT','SOLDDATE_CAT']]
unflipped_homes.insert(0, 'FLIPPABLE', 0)
control_data = pd.concat([unflipped_homes, post_flips], ignore_index=True)

print(control_data.shape)

(3788, 13)
(129597, 13)


# Combining Test and Control Data

Clean and combine the two data frames. Know we will have an unbalanced data set, and will need to account for that in our analysis.

In [13]:
flip_df = pd.DataFrame.from_dict(flip_data)
control_df = pd.DataFrame.from_dict(control_data)

In [14]:
flip_df.head(20)
print(flip_df.shape)
control_df.head(20)
print(control_df.shape)

(3788, 15)
(129597, 13)


In [15]:
# combine test and control
merged_test_and_control_data = pd.concat([flip_df, control_df], ignore_index=True)
print(merged_test_and_control_data.shape)

# remove bad listings for now
bad_data = ['72250832','71902243','72214658','72099376','72032454','72027853','72018311',
            '71955378','72045937','72133139','72144618']
merged_test_and_control_data = merged_test_and_control_data[~merged_test_and_control_data['MLSNUM'].isin(bad_data)]

# clean up data that should be ints or floats
merged_test_and_control_data[['SOLDPRICE', 'AGE']] = merged_test_and_control_data[['SOLDPRICE', 'AGE']].astype(str)

merged_test_and_control_data['SOLDPRICE'] = list(map(lambda x: float(x.replace(",", "")), merged_test_and_control_data['SOLDPRICE']))

merged_test_and_control_data['AGE'] = list(map(lambda x: int(x.replace(",", "")), merged_test_and_control_data['AGE']))
merged_test_and_control_data[['MLSNUM', 'DOM', 'GARAGE','SOLDDATE_CAT']] = merged_test_and_control_data[['MLSNUM', 'DOM', 'GARAGE', 'SOLDDATE_CAT']].astype(int)
merged_test_and_control_data['FLIPPABLE'] = merged_test_and_control_data['FLIPPABLE'].astype(bool)

# verify
merged_test_and_control_data.info()
print(merged_test_and_control_data.shape)
merged_test_and_control_data.head(20)

(133385, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133375 entries, 0 to 133384
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   FLIPPABLE     133375 non-null  bool   
 1   MLSNUM        133375 non-null  int64  
 2   SOLDPRICE     133375 non-null  float64
 3   DOM           133375 non-null  int64  
 4   ZIP           3788 non-null    float64
 5   BEDS          133375 non-null  int64  
 6   BATHS         133375 non-null  float64
 7   SQFT          133375 non-null  int64  
 8   AGE           133375 non-null  int64  
 9   LOTSIZE       3788 non-null    object 
 10  GARAGE        133375 non-null  int64  
 11  PROPTYPE_CAT  133375 non-null  int64  
 12  STYLE_CAT     133375 non-null  int64  
 13  ZIP_CAT       133375 non-null  int64  
 14  SOLDDATE_CAT  133375 non-null  int64  
dtypes: bool(1), float64(3), int64(10), object(1)
memory usage: 15.4+ MB
(133375, 15)


,FLIPPABLE,MLSNUM,SOLDPRICE,DOM,ZIP,BEDS,BATHS,SQFT,AGE,LOTSIZE,GARAGE,PROPTYPE_CAT,STYLE_CAT,ZIP_CAT,SOLDDATE_CAT
0,True,71425748,50000.0,1077,2740.0,4,2.0,1984,95,1888,0,2,458,168,735998
1,True,71477127,125000.0,724,2368.0,4,2.0,2333,67,10000,0,0,1,207,736018
2,True,71478495,720500.0,828,1532.0,4,3.5,4196,20,117176,3,0,3,313,735978
3,True,71485790,160000.0,1140,1746.0,3,1.5,1350,58,17860,0,0,6,71,736215
4,True,71614526,335000.0,809,2649.0,3,2.0,1270,38,5401,1,0,6,111,736024
5,True,71633454,364000.0,106,2127.0,4,1.5,1462,118,1250,0,0,3,10,736009
6,True,71661835,135000.0,760,2019.0,6,2.0,2315,98,12000,0,2,458,45,735983
7,True,71701065,240000.0,343,2364.0,4,2.0,1346,38,77972,0,0,1,233,735982
8,True,71716837,440000.0,42,2090.0,3,3.0,2046,81,42178,1,0,1,44,736039
9,True,71745072,215000.0,1369,1453.0,3,3.5,2112,31,18002,2,0,3,368,736048


In [16]:
#read in image count file
image_counts = pd.read_csv('./outputs/image_count.csv')
image_counts.drop('Unnamed: 0',axis=1,inplace=True)
image_counts.head()

,MLS_num,count
0,72007367,12
1,72029311,10
2,72013464,10
3,72027618,10
4,72003490,10


In [28]:
#merge test and control data with image count file on MLS num
merged_test_and_control_data = (
    merged_test_and_control_data.merge(image_counts,left_on='MLSNUM',right_on='MLS_num',how='left').
    rename(columns={'count':'IMAGES'}).
    drop('MLS_num',axis=1).
    drop_duplicates()
)
#get only rows with images
data_with_images = merged_test_and_control_data[merged_test_and_control_data['IMAGES'].notnull()]
data_with_images


In [20]:
#Include zero images; fill in NaNs and change back to integer
merged_test_and_control_data['IMAGES'].fillna(0,inplace=True)
merged_test_and_control_data['IMAGES']= merged_test_and_control_data['IMAGES'].astype('int64')
merged_test_and_control_data

,FLIPPABLE,MLSNUM,SOLDPRICE,DOM,ZIP,BEDS,BATHS,SQFT,AGE,LOTSIZE,GARAGE,PROPTYPE_CAT,STYLE_CAT,ZIP_CAT,SOLDDATE_CAT,IMAGES
0,True,71425748,50000.0,1077,2740.0,4,2.0,1984,95,1888,0,2,458,168,735998,0
1,True,71477127,125000.0,724,2368.0,4,2.0,2333,67,10000,0,0,1,207,736018,0
2,True,71478495,720500.0,828,1532.0,4,3.5,4196,20,117176,3,0,3,313,735978,0
3,True,71485790,160000.0,1140,1746.0,3,1.5,1350,58,17860,0,0,6,71,736215,4
4,True,71614526,335000.0,809,2649.0,3,2.0,1270,38,5401,1,0,6,111,736024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133087,True,72252482,370000.0,117,NaN,0,0.0,2337,30,NaN,2,2,458,141,736775,0
133126,True,72276079,695000.0,15,NaN,4,2.0,1722,73,NaN,0,0,3,63,736779,0
133167,True,72270032,380000.0,33,NaN,6,2.5,1942,246,NaN,0,2,458,264,736783,0
133340,True,72279039,267000.0,20,NaN,3,2.0,1822,13,NaN,1,0,7,569,736782,0


In [ ]:
#save merged files
data_with_images.to_csv('./outputs/data_with_images.csv')
merged_test_and_control_data.to_csv('./outputs/merged_test_and_control_data.csv')

# Profit margins

This will spit out a csv of pre and post values of the FLIPs, with a profit margin:

In [19]:
# build flip set
flips = [];
for mls in flippedHomes['MLSNUM']:
    homes = getHome(flippedHomes, mls)
    # check if home has 2 mls rows
    if(homes.shape[0] >= 2):
        pre = homes.iloc[0]
        post = homes.iloc[1]
        # check if it's not the same mls number
        if pre['MLSNUM'] == post['MLSNUM']:
            continue
        # find pre and post flip values
        if pd.to_datetime(pre['SOLDDATE']) > pd.to_datetime(post['SOLDDATE']):
            # post < pre: swap them
            swap = pre
            pre = post
            post = swap
        flips.append({
            'PRE-FLIP-MLS': pre['MLSNUM'],
            'POST-FLIP-MLS': post['MLSNUM'],
            'PRE-FLIP-SOLD-DATE': pre['SOLDDATE'],
            'POST-FLIP-SOLD-DATE': post['SOLDDATE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PROFIT': int(post['SOLDPRICE']) - int(pre['SOLDPRICE'])
        })
        
flips = pd.DataFrame.from_dict(flips)
flips.to_csv('./outputs/flips.csv')
flips.head()

,PRE-FLIP-MLS,POST-FLIP-MLS,PRE-FLIP-SOLD-DATE,POST-FLIP-SOLD-DATE,PRE-SOLD-PRICE,POST-SOLD-PRICE,PROFIT
0,71425748,72069748,02/04/2016,1/20/2017,50000,185000,135000
1,71477127,72073263,2/24/2016,11/18/2016,125000,395000,270000
2,71478495,72246808,1/15/2016,12/11/2017,720500,821000,100500
3,71485790,72098121,9/8/2016,01/12/2017,160000,320000,160000
4,71614526,72131597,03/01/2016,05/01/2017,335000,425000,90000
